## Load Data

In [1]:
import numpy as np
import xarray as xr
import dask.array as da
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from scipy.integrate import cumulative_trapezoid
from ad99py.ad99dask import AlexanderDunkerton1999Dask
from ad99py.constants import GRAV,R_DRY,C_P

## Loon Load

In [2]:
from glob import iglob 
import os
LOON_GW_DATA_PATH = 'data/loon'
u_flux_ptv = []
u_flux_ntv = []
v_flux_ntv = []
v_flux_ptv = []
for f in iglob(os.path.join(LOON_GW_DATA_PATH,'*flux.npy')):
    flux = np.load(f)
    u_flux_ntv.append(flux[0])
    u_flux_ptv.append(flux[1])
    v_flux_ntv.append(flux[2])    
    v_flux_ptv.append(flux[3])


loon_ntv_u = np.concatenate(u_flux_ntv)
loon_ptv_u = np.concatenate(u_flux_ptv)
loon_ntv_v = np.concatenate(v_flux_ntv)
loon_ptv_v = np.concatenate(v_flux_ptv)
loon_abs_v = np.concatenate([np.abs(loon_ntv_v)+np.abs(loon_ptv_v)])
loon_abs_u = np.concatenate([np.abs(loon_ntv_u)+np.abs(loon_ptv_u)])
loon_net_u = np.concatenate([loon_ntv_u+loon_ptv_u])
loon_net_v = np.concatenate([loon_ntv_v+loon_ptv_v])

LOON_DATA = {
    'u_flux_ntv': loon_ntv_u[loon_ntv_u<0],
    'u_flux_ptv': loon_ptv_u[loon_ptv_u>0],
    'v_flux_ntv': loon_ntv_v[loon_ntv_v<0],
    'v_flux_ptv': loon_ptv_v[loon_ptv_v>0],
    'u_flux_abs': loon_abs_u[loon_abs_u>0],
    'v_flux_abs': loon_abs_v[loon_abs_v>0],
    'u_flux_net': loon_net_u,
    'v_flux_net': loon_net_v
}

## Load + Mask Data

In [3]:
## Load in MiMA data
ds = xr.open_dataset('../era5_coarse_daily2015.nc')
## Load Masks
ds_mask_trop = xr.open_dataset('data/loon_masks.nc')

ds_mask_trop['lon'] = (ds_mask_trop.lon + 360) % 360 
ds_mask_trop = ds_mask_trop.sortby('lon')
ds_mask_trop = ds_mask_trop.rename(lat='latitude',lon='longitude')
interp_mask = ds_mask_trop.interp(latitude=ds.latitude,longitude=ds.longitude,method='nearest').astype(bool)
total_mask = sum(interp_mask[d] for d in interp_mask.data_vars)

# ocean_expac = ds.where(total_mask).stack(points = ['lat','lon'])
# ocean_expac = ocean_expac.dropna('points',how='all')
# ocean_expac = ocean_expac[['ucomp','temp','vcomp','height','gwfu_cgwd','gwfv_cgwd','omega']]
# ocean_expac = ocean_expac.transpose('time','points','pfull')
# ocean_expac = ocean_expac.isel(pfull=slice(None,None,-1))


# dTdz = da.gradient(ocean_expac.temp,axis=-1)/da.gradient(ocean_expac.height,axis=-1)
# Ns2 = GRAV/ocean_expac.temp*(dTdz + GRAV/C_P) 
# bflim = 1e-4 
# Ns2 = xr.where(Ns2 < bflim**2,bflim**2,Ns2)
# Ns = np.sqrt(Ns2)
# rho = (100*ocean_expac.pfull.data[None,None,:])/(R_DRY*ocean_expac.temp)

# rho = rho.data.persist()
# lat = ocean_expac.lat.data

# u = ocean_expac.ucomp.data.persist()
# z = ocean_expac.height.data.persist()
# Ns = Ns.data.persist()
# lat = da.broadcast_to(lat,(u.shape[0],u.shape[1])).persist()

In [4]:
ds_points = ds.where(total_mask).stack(points=['latitude','longitude'])
ds_points = ds_points.dropna('points',how='all')
ds_points = ds_points.transpose('time','points','level')
ds_points = ds_points.reset_index('points')


In [5]:
lat = ds_points.latitude.data

# Calibrate AD99 $c_w$ to Loon 
We can always RECENTER by changing the intermittency

In [6]:
cws = np.arange(5,80,5)
level = 70 
i_level = np.argmin(np.abs(ds_points.level.data - level))
i_level

np.int64(27)

In [7]:
u = ds_points.u.data
z = ds_points.z.data 
Ns = ds_points.Ns.data
rho = ds_points.rho.data 


numpy - columnwise

dask

In [ ]:
from dask.diagnostics import ProgressBar
from ad99py.ad99dask import AlexanderDunkerton1999Dask
uw_ptv_level = []

for c in cws:
    print(c)

    ad99 = AlexanderDunkerton1999Dask(cw=c,dc=0.4,exclude_unbroken=True,use_intrinsic_c='never')
    with ProgressBar():
        _, ptv = ad99.momentum_flux_neg_ptv(u,Ns,z,rho,lat)
        ptv_lvl = ptv[...,i_level].compute().ravel()
    uw_ptv_level.append(ptv_lvl[ptv_lvl > 0])

5
[########################################] | 100% Completed | 238.97 s
10
[########################################] | 100% Completed | 239.64 s
15
[########################################] | 100% Completed | 239.27 s
20
[################################        ] | 80% Completed | 159.18 s


KeyboardInterrupt: 